# Summary evaluation

Today we'll take a look at how we can evaluate the quality of model-generated summaries in different ways.

## Install packages

Tip: You might need to restart the jupyter kernel after installation.

In [1]:
%pip install rouge_score
%pip install bert_score 
%pip install blanc 
%pip install nltk 
%pip install sentencepiece 
%pip install protobuf 
%pip install transformers 
%pip install datasets 
%pip install spacy
%pip install evaluate
!python -m spacy download en_core_web_sm

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 335.9 kB/s eta 0:00:000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 1.3 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 1.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 5.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 17.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 kB 13.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 3.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 404.9 kB/s eta 0:00:00a 0:00:01
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=a98aee626d

## Load the data

We'll use a small slice of the English part of the `xlsum` dataset from the `datasets` library. You can take a look at what kind of data this includes [here](https://huggingface.co/datasets/csebuetnlp/xlsum).

In [1]:
from datasets import load_dataset

ds = load_dataset("csebuetnlp/xlsum", "english", split='train[:1%]')

README.md:   0%|          | 0.00/14.6k [00:00<?, ?B/s]

xlsum.py:   0%|          | 0.00/4.55k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/315M [00:00<?, ?B/s]

0001.parquet:   0%|          | 0.00/264M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/20.8M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/20.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/306522 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11535 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11535 [00:00<?, ? examples/s]

In [4]:
ds

Dataset({
    features: ['id', 'url', 'title', 'summary', 'text'],
    num_rows: 3065
})

The articles are in the `text` column and the summaries are in the `summary` column. Let's extract them and take a look at a few examples.

In [2]:
articles = ds["text"][0:10]
articles

['The Met Office has issued a yellow weather warning for wind covering Wales and England, starting from 21:00 GMT on Wednesday evening. Travel and power are both likely to be disrupted, with the warning to remain in place until 15:00 on Thursday. Gusts of 55mph (88kmh) are likely and could hit up to 70mph on coasts and hills, with heavy and blustery showers.',
 'Atlantis Resources unveiled the marine energy device at Invergordon ahead of it being shipped to Kirkwall. Trials on the device will now be run at the European Marine Energy Centre test site off Eday. The device stands 22.5m (73ft) tall, weighs 1,300 tonnes and has two sets of blades on a single unit. It could generate enough power for 1,000 homes.',
 'Police were called to the scene outside the Coral shop on Compton Road in Harehills just before 14:00 BST. The man was taken to hospital for treatment but his condition is not known. West Yorkshire Police said the area has been cordoned off and officers remain at the scene. The f

In [3]:
reference_summaries = ds["summary"][0:10]
reference_summaries

['Winds could reach gale force in Wales with stormy weather set to hit the whole of the country this week.',
 'The massive tidal turbine AK1000 has been installed in 35m (114.8ft) of water at a test site in Orkney.',
 'A man has been stabbed in broad daylight outside a betting shop in Leeds.',
 'It was inevitable that the fallout from the US airstrike that killed Iranian General Qasem Soleimani would spill into presidential politics. Everything spills into presidential politics these days, and this is without a doubt a major story.',
 'Week four of social distancing is starting to take its toll.',
 'A 37-year-old man has been arrested as part an ongoing investigation into criminality linked to the North Antrim Ulster Defence Association (UDA).',
 'Electric buses will soon be running on the roads in Coventry.',
 'A Jersey deputy is calling on the number of States members to be reduced more than current proposals.',
 'About 200 posts are to go at the Boots site in Nottingham.',
 'A degre

Discuss:
- Based on these examples, what do you think of the quality of the dataset?
- Do you foresee any potential pitfalls for evaluation, based on your observations?

Let's take a look into the density of the summaries.

In [4]:
from utils.fragments import Fragments

fragment = [Fragments(summary, article, lang="en") for summary, article in zip(reference_summaries, articles)]
density = [frag.density() for frag in fragment]

/home/ucloud/.local/lib/python3.10/site-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [8]:
len(list(filter(lambda x: x <= 1.5, density))) / len(density)

1.0

If you remember, summaries with density values below 1.5 are considered abstractive, meaning these seem to be highly abstractive summaries.
However, the density values are not a perfect measure of abstractive quality:
- Can you think of a way we might be able to "game" the density metric?

## Generating summaries
Now let's generate some summaries using a pre-trained model. We'll use the `mt5-small` model from the `transformers` library.

In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "google/mt5-small"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, min_length=10, max_length=50)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/home/ucloud/.local/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

To make everything a bit easier for ourselves, let's make a function which:
1. Takes an input text
2. Tokenises the text (remember to set the padding and truncation arguments to True)
3. Generates a summary based on the tokenised input (and prompt, if you're so inclined)
4. Decodes the generated summary from tokens into words, and
5. Returns the output

(Hint: there is one potential solution in the class_8_solution notebook, if you're in need :-)).

Now let's use that function to generate some summaries for the articles in the dataset.

In [7]:
your_pipeline_function(articles[0])

/home/ucloud/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1493: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


'<pad> <extra_id_0>.com.au./ <extra_id_10>.</s>'

In [8]:
generated_summaries = [your_pipeline_function(article) for article in articles]

In [10]:
generated_summaries

['<pad> <extra_id_0>.com.au./ <extra_id_10>.</s>',
 '<pad> <extra_id_0>.com <extra_id_10>.com an <extra_id_11>.</s>',
 '<pad> <extra_id_0>.com.uk.com <extra_id_10>.com</s>',
 '<pad> <extra_id_0> the debate. Politics.com</s>',
 '<pad> <extra_id_0> Jamie Kennedy says: "It was hard.</s>',
 '<pad> <extra_id_0>. ) <extra_id_37>.graves. <extra_id_4> - Criminal Investigations</s>',
 '<pad> <extra_id_0> - Coventry - Coventry</s>',
 '<pad> <extra_id_0> - St Helier - St Helier</s>',
 '<pad> <extra_id_0> - Consumer Reports - Business</s>',
 '<pad> <extra_id_0>.com.uk.com/courses.</s>']

## Evaluation
Now let's evaluate the quality of the generated summaries with some commonly used metrics.

In [ ]:
from evaluate import load

rouge = load("rouge")
rouge.compute(references=reference_summaries, predictions=generated_summaries)

{'rouge1': np.float64(0.02448512585812357),
 'rouge2': np.float64(0.0),
 'rougeL': np.float64(0.025069921179760996),
 'rougeLsum': np.float64(0.02448512585812357)}

We can also take a look at the ROUGE scores for the individual summaries:

In [ ]:
rouge.compute(references=reference_summaries, predictions=generated_summaries, use_aggregator=False)

{'rouge1': [0.0,
  0.0,
  0.0,
  0.08695652173913043,
  0.0,
  0.052631578947368425,
  0.1111111111111111,
  0.0,
  0.0,
  0.0],
 'rouge2': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 'rougeL': [0.0,
  0.0,
  0.0,
  0.08695652173913043,
  0.0,
  0.052631578947368425,
  0.1111111111111111,
  0.0,
  0.0,
  0.0],
 'rougeLsum': [0.0,
  0.0,
  0.0,
  0.08695652173913043,
  0.0,
  0.052631578947368425,
  0.1111111111111111,
  0.0,
  0.0,
  0.0]}

The BERTScore metric does not use an aggregator, but we can average the scores ourselves to get an overall score.

In [ ]:
bertscore = load("bertscore")
bertscores = bertscore.compute(references=reference_summaries, predictions=generated_summaries, lang="en")
bertscores

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'precision': [0.7897380590438843,
  0.7658491730690002,
  0.7976936101913452,
  0.7899141907691956,
  0.8089547157287598,
  0.7626368999481201,
  0.8046824932098389,
  0.7890907526016235,
  0.7653212547302246,
  0.8175322413444519],
 'recall': [0.8295823931694031,
  0.7967324256896973,
  0.826530396938324,
  0.8113272190093994,
  0.8521971702575684,
  0.8178002834320068,
  0.873009204864502,
  0.8301193714141846,
  0.8252494931221008,
  0.8258838653564453],
 'f1': [0.8091700673103333,
  0.7809856534004211,
  0.8118560910224915,
  0.8004775047302246,
  0.83001309633255,
  0.7892558574676514,
  0.8374544978141785,
  0.8090853095054626,
  0.7941564321517944,
  0.8216868042945862],
 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.46.3)'}

In [16]:
import numpy as np

np.mean(bertscores["precision"]), np.mean(bertscores["recall"]), np.mean(bertscores["f1"])

(np.float64(0.7891413390636444),
 np.float64(0.8288431823253631),
 np.float64(0.8084141314029694))

We can also try a reference-free metric, such as BLANC, in case we do not have access to reference summaries, or we do not want to rely on them due to quality, etc.

In [ ]:
import blanc

blanc = blanc.BlancHelp()
blanc.eval_pairs(articles, generated_summaries)

ModuleNotFoundError: No module named 'blanc'

Discuss:
- What do these values tell us about the quality of the generated summaries?
- What are the strenghts and weaknesses of using reference-free metrics?
- What are the potential weaknesses of using a less known metric?

## Exercise

Now, the summaries we generated aren't exactly great, likely because the mt5 model was not fine-tuned for that purpose.
- Try to generate 10 new summaries using a model that has been fine-tuned for summarisation (e.g., our old friend, flan-t5-small)
- When you have the summaries, evaluate them using the same quantitative metrics as before
- Then try to conduct a qualitative evaluation of the summaries - in your groups, decide on some evalaution criteria (e.g., ranking, "stars", etc.), evaluate the summaries based on these criteria, and compare your results within the group and with the quantitative metrics

### Bonus exercise
Try to create a LLM judge that can evaluate the quality of the summaries based on the criteria you defined.
- Load in a generative pre-trained model from huggingface
- Prompt it with your evaluation criteria
- Compare its evaluation with your own

In [62]:
import nltk

nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/au594328/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True